In [1]:
import pandas as pd
import plotly.express as px
import re
import datetime

from Project.Database import Db

In [2]:
year1_hourly, meta = Db.load_data(hourly=True, meta=True, year=1)

In [3]:
consumption_condition = (lambda self: (
            (self["Units"] == "W") &
            self["Description"].str.contains("power consumption" or "used") &
            ~(self["Unnamed: 0"] == "Elec_PowerMicrowave") &
            ~(self["Unnamed: 0"] == "Elec_PowerRefrigerator") &
            ~(self["Unnamed: 0"] == "Elec_PowerClothesWasher") &
            ~self["Unnamed: 0"].str.contains("Elec_[\w]+HVAC") &
            (~self["Unnamed: 0"].str.contains("DHW_") |
             self["Unnamed: 0"].str.contains("DHW_[\w]+Total")) &
            (~self["Unnamed: 0"].str.contains("Load_") |
             ~self["Unnamed: 0"].str.contains("Load_Microwave") &
             self["Unnamed: 0"].str.contains("Load_[\w]+Standby"))))

consumption = meta.loc[consumption_condition]["Unnamed: 0"].tolist()